### Data Dictionary

A. track_record_id: Unique number given to each log

B. status_code: Log correctness code (61445 indicates good log, all other    status code indicate bad)  

C. latitude_gps: latitude of the equipment at the given time

D. longitude_gps: longitude of the equipment at the given time

E. device_id_x: Equipment identifier (used by team x) 

F. device_time_stamp: time (datetime) when log is created

G. asset_type: 4xxx = movable equipment type 1; 6xxx = stationary equipment type 1;  8xxx = movable equipment type 2;    7xxx = stationary equipment type 2

H. packet_generating_station_id: Equipment identifier (to be used by internship candidate)

I. track_num1: Indicates whether equipment ignition is ON (1) or OFF (0)

J. asset_id_project: Equipment identifier (used by team y)

K. project_id: project number where the equipment is working at the given time

L. supportsite_id: segment of project to which the equipment is assigned to


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Data_for_intern_project_1.csv")

/Users/yadunandan/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3,4,6,7,8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.shape

(236343, 12)

In [5]:
#transpose of top 5 columns to view the data 
df.head().T

,0,1,2,3,4
track_record_id,59231753,59231858,59231969,59232069,59232173
status_code,61441,61441,61441,61441,61441
latitude_gps,0,0,0,0,0
longitude_gps,0,0,0,0,0
device_id_x,1001020218,1001020218,1001020218,1001020218,1001020218
device_time_stamp,12/13/2017 0:00,12/13/2017 0:01,12/13/2017 0:02,12/13/2017 0:03,12/13/2017 0:04
asset_type,4111,4111,4111,4111,4111
packet_generating_station_id,411100000001,411100000001,411100000001,411100000001,411100000001
track_num1,0,0,0,0,0
asset_id_project,MPV/27,MPV/27,MPV/27,MPV/27,MPV/27


In [7]:
# 61445 indicates good log
# let us filter only good logs
df2 = df[df["status_code"] == 61445].copy()

In [9]:
# Equipment identifier (to be used by internship candidate)
df2["packet_generating_station_id"].nunique()

27

In [13]:
import geopy.distance
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.vincenty(coords_1, coords_2).km)

279.35290160386563


/Users/yadunandan/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


In [43]:
# Time stamp to pandas datetime data type
df2['device_time_stamp'] = pd.to_datetime(df2['device_time_stamp'])

In [44]:
#removing time stamp to calculate distnce on daily basis
df2['date'] = df2['device_time_stamp'].dt.date

#### Bring Logitude and lattitude in a single column as tuple

In [18]:
df2['lat_long'] = df[['latitude_gps', 'longitude_gps']].apply(tuple, axis=1)

In [20]:
df2['lat_long'].head()

196608    (28.19039917, 77.42134857)
196609     (28.19041061, 77.4213562)
196610    (28.19039917, 77.42134857)
196611    (28.19039917, 77.42134857)
196612    (28.19039917, 77.42134857)
Name: lat_long, dtype: object

In [46]:
def conv_list(series):
    return tuple(series)

In [47]:
# Group by on Equipment ID and date to get Daily distance travelled
df3 = df2.groupby(['packet_generating_station_id','date']).agg({'lat_long':conv_list}).reset_index()

In [48]:
df3.head()

,packet_generating_station_id,date,lat_long
0,401100000001,2017-12-17,"((28.14563942, 77.38845062), (28.14537048, 77...."
1,403100000048,2017-12-17,"((28.19039917, 77.42134857), (28.19041061, 77...."
2,404100000005,2017-12-17,"((28.14064026, 77.37538147), (28.14064026, 77...."
3,405100000003,2017-12-17,"((28.14146042, 77.37531281), (28.14145088, 77...."
4,406100000024,2017-12-17,"((28.22020912, 77.43312073), (28.22020912, 77...."


#### A. Distance traveled: total distance (unit: km) covered by each equipment on daily basis

In [50]:

# function to convert longitude and latitude to distance in km
from math import sin, cos, sqrt, atan2, radians

def dist_calc(series):
    dist_sum = 0
    
    # approximate radius of earth in km
    R = 6373.0
    for i in range(0,len(series)-1):
        
        coords_1 = series[i]
        coords_2 = series[i+1]
        lat1 = radians(coords_1[0])
        lon1 = radians(coords_1[1])
        lat2 = radians(coords_2[0])
        lon2 = radians(coords_2[1])

        dlon = lon2 - lon1
        dlat = lat2 - lat1
        
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c
        
        dist_sum += distance
        
    return dist_sum

In [51]:
df3["distance"] = df3['lat_long'].apply(lambda x: dist_calc(x))

In [57]:
# Distance travelled on daily basis
df3.head()

,packet_generating_station_id,date,lat_long,distance
0,401100000001,2017-12-17,"((28.14563942, 77.38845062), (28.14537048, 77....",21.291156
1,403100000048,2017-12-17,"((28.19039917, 77.42134857), (28.19041061, 77....",0.218827
2,404100000005,2017-12-17,"((28.14064026, 77.37538147), (28.14064026, 77....",4.523435
3,405100000003,2017-12-17,"((28.14146042, 77.37531281), (28.14145088, 77....",2.001275
4,406100000024,2017-12-17,"((28.22020912, 77.43312073), (28.22020912, 77....",0.638775


#### 2. B. ON time: total time (unit: hour) for which ignition of each equipment was ON (track_num1 = 1)

In [59]:
#trck on =1

df4 = df2[['packet_generating_station_id','device_time_stamp','track_num1']][df2['track_num1'] == 1]

In [60]:
df4.shape

(3641, 3)

In [79]:
df4

,packet_generating_station_id,device_time_stamp,track_num1
197282,801100000024,2017-12-17 08:02:00,1
197283,801100000024,2017-12-17 08:03:00,1
197284,801100000024,2017-12-17 08:04:00,1
197285,801100000024,2017-12-17 08:05:00,1
197286,801100000024,2017-12-17 08:06:00,1
197287,801100000024,2017-12-17 08:07:00,1
197288,801100000024,2017-12-17 08:08:00,1
197289,801100000024,2017-12-17 08:09:00,1
197304,801100000024,2017-12-17 08:24:00,1
197305,801100000024,2017-12-17 08:25:00,1


In [89]:
#Convert timestamp to seconds
df4['time_seconds'] = df4['device_time_stamp'].apply(lambda x: (x-datetime(1970,1,1,0,0,0)).total_seconds())

In [98]:
df5 = df4.groupby(['packet_generating_station_id']).agg({'time_seconds':conv_list}).reset_index()

In [100]:
df5.shape

(18, 2)

In [102]:
def on_time(series):
    total_time = 0
    
    for i in range(0,len(series)-1):
        t1 = series[i]
        t2 = series[i+1]
    
        total_time += abs(t1-t2)/ 3600.0
        
    return total_time



In [104]:
df5["on_time"] = df5['time_seconds'].apply(lambda x: on_time(x))

In [106]:
df5[['packet_generating_station_id','on_time']]

,packet_generating_station_id,on_time
0,408300000002,8.783333
1,409100000006,1.833333
2,410100000004,9.916667
3,415200000003,1.633333
4,701800000007,13.666667
5,801100000024,10.500000
6,801100000035,9.116667
7,801100000060,9.700000
8,801100000101,9.300000
9,802100000001,9.533333


#### 3. C. Equipments in the vicinity: if a main equipment is at the location of (latitude = 28.09029007, longitude = 77.33970642), provide a map (one map for each day) showing:
#### a. geofence/circle of radius=100m around the main equipment; 
#### b. Equipments present inside fence between 9:00am and 9:10am (one map each day)

In [115]:
df2['latitude_gps'] = df2['latitude_gps'].astype(float)
df2['longitude_gps'] = df2['longitude_gps'].astype(float)

In [150]:
df2[(df2['latitude_gps'] == 28.09029007)&(df2['longitude_gps'] == 77.33970642)]

,track_record_id,status_code,latitude_gps,longitude_gps,device_id_x,device_time_stamp,asset_type,packet_generating_station_id,track_num1,asset_id_project,project_id,supportsite_id,lat_long,date
